In [2]:
import numpy as np
import torch 
import cv2
import torch.nn as nn 
import pandas as pd
import os
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
import time
import torch.nn.functional as F
import torch.optim as optim 

In [76]:
# training 時做 data augmentation
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), # 隨機將圖片水平翻轉
    transforms.RandomRotation(15), # 隨機旋轉圖片
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] (data normalization)
])
# testing 時不需做 data augmentation
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label is required to be a LongTensor
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y.numpy())
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [4]:
training_data = np.load("training_data_128.npy", allow_pickle = True)
training_data.shape

(24946, 2)

In [52]:
X = torch.Tensor([i[0] for i in training_data]).view(-1, 128, 128)
# Scaling the features
X = X/255.0
# Getting the target
y = torch.Tensor([i[1] for i in training_data])

In [53]:
print(y.shape)

torch.Size([24946, 2])


In [61]:
y_label = []
real = torch.argmax(y)
for i in range(len(y)):
    real = torch.argmax(y[i])
    y_label.append(real.item())


In [78]:
print(len(y_label))
count = 0
for i in range(len(y_label)):
    if y_label[i] ==0:
        count+=1
#功有 12476 猫 24946- 12476 只狗
print(count)
y_label = torch.Tensor(y_label)

24946
12476


In [79]:
val = int(len(X)*0.3)
train_x = X[:-val]
train_y = y_label[:-val]
val_x = X[-val:]
val_y = y_label[-val:]

In [93]:
batch_size=64
train_set = ImgDataset(train_x,train_y,train_transform)
test_set = ImgDataset(val_x,val_y,test_transform)
train_loader = DataLoader(train_set,batch_size = batch_size,shuffle=True)
test_loader = DataLoader(test_set,batch_size=batch_size,shuffle=False)

In [81]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [1, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128), 
            nn.ReLU(), 
            nn.Linear(128,2)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [91]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
# print(device)
model = Classifier().to(device)
cirection = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
# model.eval()

In [94]:
#train
# model.train()
epochs = 80
for epoch in range(epochs):
    epoch_start_time = time.time()
    train_acc =0.0
    val_acc =0.0
    train_loss = 0.0
    val_loss =0.0
    model.train()
    for i,data in enumerate(train_loader):
        optimizer.zero_grad()
        x,y = data[0].to(device),data[1].to(device)
        # print(x.shape)
        y_pred = model(x)
        loss = cirection(y_pred,y.long())
        loss.backward()
        optimizer.step()
        train_acc +=np.sum(np.argmax(y_pred.cpu().data.numpy(),axis=1)== y.cpu().numpy())
        train_loss +=loss.item()
    model.eval()
    with torch.no_grad():
        for i,data in enumerate(test_loader):
            valx ,valy = data[0].to(device),data[1].to(device)
            val_pred = model(valx)
            batch_loss = cirection(val_pred,valy.long())
            val_acc +=np.sum(np.argmax(val_pred.cpu().data.numpy(),axis=1)== valy.cpu().numpy())
            val_loss +=batch_loss.item()

        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, epochs, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/test_set.__len__(), val_loss/test_set.__len__()))

torch.save(model,'cat_dog_128.pth')

[001/080] 72.86 sec(s) Train Acc: 0.724618 Loss: 0.008468 | Val Acc: 0.755980 loss: 0.007630
[002/080] 72.00 sec(s) Train Acc: 0.792075 Loss: 0.006937 | Val Acc: 0.763731 loss: 0.007292
[003/080] 71.56 sec(s) Train Acc: 0.840634 Loss: 0.005609 | Val Acc: 0.858613 loss: 0.005121
[004/080] 72.03 sec(s) Train Acc: 0.872301 Loss: 0.004684 | Val Acc: 0.784177 loss: 0.007915
[005/080] 71.45 sec(s) Train Acc: 0.892802 Loss: 0.003952 | Val Acc: 0.891888 loss: 0.004112
[006/080] 72.53 sec(s) Train Acc: 0.909179 Loss: 0.003407 | Val Acc: 0.918883 loss: 0.003059
[007/080] 74.17 sec(s) Train Acc: 0.920060 Loss: 0.003055 | Val Acc: 0.915943 loss: 0.003222
[008/080] 74.17 sec(s) Train Acc: 0.926072 Loss: 0.002798 | Val Acc: 0.929574 loss: 0.002639
[009/080] 74.13 sec(s) Train Acc: 0.932314 Loss: 0.002534 | Val Acc: 0.907390 loss: 0.003560
[010/080] 74.10 sec(s) Train Acc: 0.939415 Loss: 0.002311 | Val Acc: 0.932514 loss: 0.002798
[011/080] 74.11 sec(s) Train Acc: 0.944282 Loss: 0.002135 | Val Acc: 0

In [95]:
print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, epochs, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/test_set.__len__(), val_loss/test_set.__len__()))

[080/080] 23308.28 sec(s) Train Acc: 0.996507 Loss: 0.000189 | Val Acc: 0.954831 loss: 0.003675
